In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')
df = pd.read_excel("OnlineRetail.xlsx")




df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


##Data Cleaning

In [3]:

df.info()

print("\nMissing values per column:\n")
print(df.isnull().sum())

df.dropna(subset=['CustomerID', 'Description'], inplace=True)

# Remove rows with negative Quantity or UnitPrice (likely returns or errors)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Remove canceled invoices — they usually start with 'C'
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Create TotalPrice column for each transaction (Quantity * Unit Price)
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

df.info()
  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB

Missing values per column:

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (tot

In [4]:
# Convert 'InvoiceDate' to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print("Snapshot Date:", snapshot_date)

# Group by CustomerID and calculate RFM metrics
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,   # Recency
    'InvoiceNo': 'nunique',                                     # Frequency (number of orders)
    'TotalPrice': 'sum'                                         # Monetary (total spent)
}).reset_index()

# Rename columns
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

rfm.head()


Snapshot Date: 2011-12-10 12:50:00


,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40


In [5]:
# Recency score: lower days = higher score
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])

# Frequency score: higher orders = higher score
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])

# Monetary score: higher spend = higher score
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

# Combine into a single RFM Segment
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Calculate overall RFM Score (sum of individual scores)
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].astype(int).sum(axis=1)

rfm.head()


,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment,RFM_Score
0,12346.0,326,1,77183.60,1,1,4,114,6
1,12347.0,2,7,4310.00,4,4,4,444,12
2,12348.0,75,4,1797.24,2,3,4,234,9
3,12349.0,19,1,1757.55,3,1,4,314,8
4,12350.0,310,1,334.40,1,1,2,112,4


In [6]:
def segment_customer(row):
    if row['RFM_Score'] >= 10:
        return 'Champions'
    elif row['RFM_Score'] >= 8:
        return 'Loyal Customers'
    elif row['RFM_Score'] >= 6:
        return 'Potential Loyalist'
    elif row['RFM_Score'] >= 4:
        return 'Need Attention'
    else:
        return 'At Risk'

rfm['Customer_Segment'] = rfm.apply(segment_customer, axis=1)
rfm.head()


,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment,RFM_Score,Customer_Segment
0,12346.0,326,1,77183.60,1,1,4,114,6,Potential Loyalist
1,12347.0,2,7,4310.00,4,4,4,444,12,Champions
2,12348.0,75,4,1797.24,2,3,4,234,9,Loyal Customers
3,12349.0,19,1,1757.55,3,1,4,314,8,Loyal Customers
4,12350.0,310,1,334.40,1,1,2,112,4,Need Attention


##Visualization

In [12]:
import pandas as pd
import numpy as np
import plotly.express as px

# ==== 1. Customer Segment Distribution ====
seg_counts = rfm['Customer_Segment'].value_counts().reset_index()
seg_counts.columns = ['Customer_Segment', 'count']

fig1 = px.bar(
    seg_counts,
    x='Customer_Segment',
    y='count',
    text='count',
    color='Customer_Segment',
    color_discrete_sequence=px.colors.sequential.Aggrnyl,
    title="Customer Segments Distribution"
)
fig1.update_traces(textposition='outside')
fig1.update_layout(
    xaxis_title="Customer Segment",
    yaxis_title="Number of Customers",
    title_font=dict(size=22, color='cyan'),
    plot_bgcolor="black", paper_bgcolor="black",
    font=dict(color="white")
)
fig1.show()

# ==== 2. Monetary Distribution (Boxplot with log scale) ====
fig2 = px.box(
    rfm, x="Customer_Segment", y="Monetary", color="Customer_Segment",
    color_discrete_sequence=px.colors.sequential.Inferno,
    title="Monetary Distribution by Segment (Log Scale)"
)
fig2.update_yaxes(type="log")
fig2.update_layout(
    plot_bgcolor="black", paper_bgcolor="black",
    font=dict(color="white"), title_font=dict(size=22, color='orange')
)
fig2.show()

# ==== 3. Average RFM Metrics by Segment ====
rfm_summary = rfm.groupby('Customer_Segment')[['Recency','Frequency','Monetary']].mean().reset_index()
rfm_summary_melted = rfm_summary.melt(id_vars='Customer_Segment', var_name='Metric', value_name='Average')

fig3 = px.bar(
    rfm_summary_melted, x="Customer_Segment", y="Average",
    color="Metric", barmode="group",
    color_discrete_sequence=px.colors.sequential.Plasma,
    title="Average RFM Metrics by Segment"
)
fig3.update_layout(
    plot_bgcolor="black", paper_bgcolor="black",
    font=dict(color="white"), title_font=dict(size=22, color='lime')
)
fig3.show()

# ==== 4. Heatmap: Customer Count (R vs F) ====
rfm_count = rfm.groupby(['R_Score','F_Score']).size().reset_index(name="count")

fig4 = px.density_heatmap(
    rfm_count, x="F_Score", y="R_Score", z="count",
    text_auto=True, color_continuous_scale="Turbo",
    title="Customer Count Heatmap (R vs F)"
)
fig4.update_layout(
    plot_bgcolor="black", paper_bgcolor="black",
    font=dict(color="white"), title_font=dict(size=22, color='magenta')
)
fig4.show()


C:\Users\PMYLS\AppData\Local\Temp\ipykernel_22452\1808204546.py:58: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [11]:
pip install plotly


  Using cached plotly-6.3.0-py3-none-any.whl.metadata (8.5 kB)
Using cached plotly-6.3.0-py3-none-any.whl (9.8 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
